In [2]:
'''
Goal: Create evaluation files train.txt and test.txt in the L2R LETOR File Format
https://www.microsoft.com/en-us/research/project/mslr/
In short
label qid:<queryid> <features of any number of dims>
'''
import pandas as pd
import numpy as np
product_stats = pd.read_pickle("product_stats.pkl")
query_item = pd.read_pickle("query_item.pkl")
queries_input = pd.read_pickle("queries_input.pkl")

In [3]:
#Explore the filtered data
query_item.head()

,queryId,itemId,vector_gensim_query,Relevance,is.test
0,1,7518,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
1,1,71,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
2,1,30311,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
3,1,7837,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
4,1,30792,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False


In [4]:
#Check relevance labels in test part
yy=query_item[query_item['is.test']==True]
print(yy.shape)
yy[yy['Relevance']==2].shape
yy[yy['Relevance']==1].shape
#print('Query ids from 1 to',query_item['queryId'].max())
#print(query_item.shape)

(314243, 5)


(0, 5)

In [5]:
product_stats.head()

,itemId,item_show_count,Item_clicks,Item_view_count,Item_purchase_count,userShow,userView,userPurchase,CTR,View Rate,Click Value Rate,vector_gensim_product
0,1,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.0075825127, 0.0077215955, -0.0076009654, 0...."
1,2,3.0,36.0,41.0,2.0,2.0,1.0,1.0,12.0,13.666667,0.666667,"[0.002787637, 0.0003684413, -0.007916889, -0.0..."
2,3,NaN,34.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.0016407266, -0.009007929, 0.009244843, 0.00..."
3,4,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[0.007388427, -0.0066472264, 0.0064146067, 0.0..."
4,6,1.0,13.0,9.0,NaN,1.0,1.0,1.0,13.0,9.000000,NaN,"[0.004354867, 0.0013623534, -0.008957111, -0.0..."


In [7]:
#For now, use only query ids from 1 to 10000 to keep the o/p file small
query_item.head()
query_item_small=query_item[query_item["queryId"]<10000]
print('Query ids range is from 1 to',query_item_small['queryId'].max())
print(query_item_small.shape)

Query ids range is from 1 to 9999
(188753, 5)


In [8]:
#Test and Train Split up
train_query_item=query_item_small[query_item_small["is.test"]==False]
test_query_item=query_item_small[query_item_small["is.test"]==True]
train_query_item.head()

,queryId,itemId,vector_gensim_query,Relevance,is.test
0,1,7518,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
1,1,71,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
2,1,30311,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
3,1,7837,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False
4,1,30792,"[-0.006535598, 0.0024064586, 0.0074732024, -0....",0,False


In [9]:
#Merge train_query_item df with product df on itemId to get all features in one df
merged_data=train_query_item.merge(product_stats, left_on='itemId', right_on='itemId')
#Confirm merge worked correctly merged.shape==train_query_item
merged_data.shape
#merged_data.head()

(123037, 16)

In [11]:
#Keep only some features for first iteration store it in temp df
temp=merged_data[['Relevance', 'queryId','Item_purchase_count','Item_clicks','userPurchase','vector_gensim_query','vector_gensim_product']].fillna(0)
temp=temp.sort_values(by=['queryId'])
temp.head()

,Relevance,queryId,Item_purchase_count,Item_clicks,userPurchase,vector_gensim_query,vector_gensim_product
0,0,1,0.0,70.0,1.0,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[0.009888342, 0.006329806, 0.0005410195, -0.00..."
1021,0,1,1.0,93.0,1.0,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[0.0031364255, 0.008238299, 0.002596087, 0.006..."
1020,1,1,0.0,44.0,1.0,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.009949418, 0.009723514, -0.0059560845, -0...."
1007,0,1,4.0,175.0,1.0,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.002778651, 4.2518728e-05, 0.008853128, -0...."
987,0,1,5.0,443.0,1.0,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.0057405643, 0.004457477, -0.001339, -0.007..."


In [12]:
#Scale/Normalise features. This needs to be checked for now /=max()
#Scale first three features
temp["Item_clicks"]=temp["Item_clicks"]/temp['Item_clicks'].max()
temp["Item_purchase_count"]=temp["Item_purchase_count"]/temp['Item_purchase_count'].max()
temp["userPurchase"]=temp["userPurchase"]/temp['userPurchase'].max()
temp.head()

,Relevance,queryId,Item_purchase_count,Item_clicks,userPurchase,vector_gensim_query,vector_gensim_product
0,0,1,0.000000,0.052711,0.25,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[0.009888342, 0.006329806, 0.0005410195, -0.00..."
1021,0,1,0.035714,0.070030,0.25,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[0.0031364255, 0.008238299, 0.002596087, 0.006..."
1020,1,1,0.000000,0.033133,0.25,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.009949418, 0.009723514, -0.0059560845, -0...."
1007,0,1,0.142857,0.131777,0.25,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.002778651, 4.2518728e-05, 0.008853128, -0...."
987,0,1,0.178571,0.333584,0.25,"[-0.006535598, 0.0024064586, 0.0074732024, -0....","[-0.0057405643, 0.004457477, -0.001339, -0.007..."


In [13]:
#Create the train file. Unpack embedding features, drop old cols
temp['queryId'] = 'qid:' + temp['queryId'].astype(str)
temp[[str(i) for i in range(50)]] = pd.DataFrame(temp.vector_gensim_query.values.tolist(), index= temp.index)
temp[[str(i+50) for i in range(50)]] = pd.DataFrame(temp.vector_gensim_product.values.tolist(), index= temp.index)
temp = temp.drop(["vector_gensim_query", "vector_gensim_product"], axis=1)
#Rounding off to 4 decimal places
temp=temp.round(4)
temp.head()

,Relevance,queryId,Item_purchase_count,Item_clicks,userPurchase,0,1,2,3,4,...,90,91,92,93,94,95,96,97,98,99
0,0,qid:1,0.0000,0.0527,0.25,-0.0065,0.0024,0.0075,-0.0028,-0.0005,...,-0.0014,0.0063,0.0075,-0.0011,-0.0037,0.0050,-0.0096,0.0071,0.0061,-0.0068
1021,0,qid:1,0.0357,0.0700,0.25,-0.0065,0.0024,0.0075,-0.0028,-0.0005,...,0.0068,0.0027,0.0083,-0.0089,0.0040,-0.0091,-0.0071,0.0098,0.0042,-0.0059
1020,1,qid:1,0.0000,0.0331,0.25,-0.0065,0.0024,0.0075,-0.0028,-0.0005,...,-0.0064,0.0028,0.0097,-0.0052,-0.0091,-0.0040,-0.0038,0.0099,-0.0064,-0.0087
1007,0,qid:1,0.1429,0.1318,0.25,-0.0065,0.0024,0.0075,-0.0028,-0.0005,...,0.0072,0.0084,-0.0006,-0.0075,-0.0026,-0.0086,-0.0086,-0.0060,-0.0018,0.0065
987,0,qid:1,0.1786,0.3336,0.25,-0.0065,0.0024,0.0075,-0.0028,-0.0005,...,0.0061,0.0061,-0.0017,0.0020,0.0098,-0.0055,0.0083,-0.0049,0.0087,-0.0095


In [14]:
#Add Tags to feature cols for creating target file format
#Time consuming #wait about half a min
feature_cols=temp.columns.tolist()[2:]
for idx,feature in enumerate(feature_cols):
    temp[feature] = str(idx+1)+':'+temp[feature].astype(str)
#Train Data Snapshot
temp.head()

,Relevance,queryId,Item_purchase_count,Item_clicks,userPurchase,0,1,2,3,4,...,90,91,92,93,94,95,96,97,98,99
0,0,qid:1,1:0.0,2:0.0527,3:0.25,4:-0.0065,5:0.0024,6:0.0075,7:-0.0028,8:-0.0005,...,94:-0.0014,95:0.0063,96:0.0075,97:-0.0011,98:-0.0037,99:0.005,100:-0.0096,101:0.0071,102:0.0061,103:-0.0068
1021,0,qid:1,1:0.0357,2:0.07,3:0.25,4:-0.0065,5:0.0024,6:0.0075,7:-0.0028,8:-0.0005,...,94:0.0068,95:0.0027,96:0.0083,97:-0.0089,98:0.004,99:-0.0091,100:-0.0071,101:0.0098,102:0.0042,103:-0.0059
1020,1,qid:1,1:0.0,2:0.0331,3:0.25,4:-0.0065,5:0.0024,6:0.0075,7:-0.0028,8:-0.0005,...,94:-0.0064,95:0.0028,96:0.0097,97:-0.0052,98:-0.0091,99:-0.004,100:-0.0038,101:0.0099,102:-0.0064,103:-0.0087
1007,0,qid:1,1:0.1429,2:0.1318,3:0.25,4:-0.0065,5:0.0024,6:0.0075,7:-0.0028,8:-0.0005,...,94:0.0072,95:0.0084,96:-0.0006,97:-0.0075,98:-0.0026,99:-0.0086,100:-0.0086,101:-0.006,102:-0.0018,103:0.0065
987,0,qid:1,1:0.1786,2:0.3336,3:0.25,4:-0.0065,5:0.0024,6:0.0075,7:-0.0028,8:-0.0005,...,94:0.0061,95:0.0061,96:-0.0017,97:0.002,98:0.0098,99:-0.0055,100:0.0083,101:-0.0049,102:0.0087,103:-0.0095


In [16]:
#Write to the train file
###################################################################
temp.to_csv(r'train.txt', header=None, index=None, sep=' ', mode='a')

In [17]:
#Repeat above steps for creating test file in a similar format

merged_data_test=test_query_item.merge(product_stats, left_on='itemId', right_on='itemId')
temp=merged_data_test[['Relevance', 'queryId','Item_purchase_count','Item_clicks','userPurchase','vector_gensim_query','vector_gensim_product']].fillna(0)
temp=temp.sort_values(by=['queryId'])
temp["Item_clicks"]=temp["Item_clicks"]/temp['Item_clicks'].max()
temp["Item_purchase_count"]=temp["Item_purchase_count"]/temp['Item_purchase_count'].max()
temp["userPurchase"]=temp["userPurchase"]/temp['userPurchase'].max()
temp['queryId'] = 'qid:' + temp['queryId'].astype(str)
temp[[str(i) for i in range(50)]] = pd.DataFrame(temp.vector_gensim_query.values.tolist(), index= temp.index)
temp[[str(i+50) for i in range(50)]] = pd.DataFrame(temp.vector_gensim_product.values.tolist(), index= temp.index)
temp = temp.drop(["vector_gensim_query", "vector_gensim_product"], axis=1)
temp=temp.round(4)
feature_cols=temp.columns.tolist()[2:]
for idx,feature in enumerate(feature_cols):
    temp[feature] = str(idx+1)+':'+temp[feature].astype(str)

In [ ]:
#Write to the test file
temp.to_csv(r'test.txt', header=None, index=None, sep=' ', mode='a')

In [18]:
#Test data snapshot
temp.head()

,Relevance,queryId,Item_purchase_count,Item_clicks,userPurchase,0,1,2,3,4,...,90,91,92,93,94,95,96,97,98,99
0,0,qid:3,1:0.0714,2:0.0218,3:0.25,4:-0.0005,5:-0.0025,6:-0.0096,7:0.0047,8:-0.006,...,94:0.0019,95:-0.0021,96:0.0026,97:-0.0091,98:-0.0076,99:0.0044,100:0.0097,101:0.0002,102:0.0087,103:-0.0015
955,0,qid:3,1:0.0,2:0.0324,3:0.25,4:-0.0005,5:-0.0025,6:-0.0096,7:0.0047,8:-0.006,...,94:0.0015,95:0.0089,96:0.0095,97:0.0031,98:0.0014,99:0.0017,100:0.0075,101:0.0067,102:-0.0083,103:0.0064
932,0,qid:3,1:0.2143,2:0.0813,3:0.25,4:-0.0005,5:-0.0025,6:-0.0096,7:0.0047,8:-0.006,...,94:0.0027,95:0.0087,96:0.0035,97:-0.0024,98:0.0074,99:-0.0025,100:0.0065,101:0.002,102:0.0066,103:0.0068
929,0,qid:3,1:0.0,2:0.0286,3:0.25,4:-0.0005,5:-0.0025,6:-0.0096,7:0.0047,8:-0.006,...,94:0.0089,95:0.0013,96:-0.0007,97:-0.0076,98:-0.0017,99:0.0048,100:-0.0088,101:0.0018,102:-0.01,103:0.0071
922,0,qid:3,1:0.1786,2:0.131,3:0.25,4:-0.0005,5:-0.0025,6:-0.0096,7:0.0047,8:-0.006,...,94:0.0037,95:0.0029,96:-0.0008,97:0.0046,98:0.0032,99:-0.0073,100:-0.0092,101:-0.0018,102:0.0025,103:0.0086


In [35]:
'''
Processed data into Aggregated LETOR/RankLib format, uploaded to google drive. Folder= Evaluation Train and Test
Each line of the data is of the follwing format:
label qid:<queryid> <features of any number of dims> (right now feature dim=103)

For now only a few features considered for the first iteration ie purchase count, clicks, user purchase and embeddings.
Data is split into train and test.
The test data is not final, relavance labels are mostly zero, this needs to be updated using provided IDCG values.

Next steps:
Use RankLib/pairwise loss regressor on this data, for now will focus on loss reduction per epoch on train data only.
'''

In [ ]:
#Ignore everything below
# d1 = {'teams': [['SF', 'NYG'],['SF', 'NYG'],['SF', 'NYG'],
#                 ['SF', 'NYG'],['SF', 'NYG'],['SF', 'NYG'],['SF', 'NYG']]
#      }
# df2=pd.DataFrame(d1)
# print (df2)
# df2[[str(i) for i in range(2)]] = pd.DataFrame(df2.teams.values.tolist(), index= df2.index)
# print(df2)
# df3=pd.DataFrame(df2['teams'].values.tolist(), columns=['team1','team2'])

In [ ]:
# with open('train.txt','r') as f:
#     for line in f:
#         tokens=line.split()
#         qid = int(tokens[1].split(":")[1])
        
# print(qid)
# x=pd.read_csv('train.txt',header=None,sep=' ')
# x.head()